Importing Packages

In [201]:
#!pip3 install imblearn

     |████████████████████████████████| 163 kB 46 kB/s eta 0:00:01


In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from bs4 import BeautifulSoup
import logging

import re
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

Importing Dataset

In [208]:
rindia = pd.read_csv('all_rIndiaScrape2.csv')

In [209]:
rindia.head()

,id,title,content,is_original_content,author,flair,score,timestamp,comments_count,crossposts_count,comments,removed_by,self_posted,nsfw,is_stickied,url
0,g0lfy8,Late Night Random Discussion Thread !,"^Beep ^Boop ^Bot, ^I ^am ^a ^bot! ^if ^any ^pr...",False,oxythebot,Scheduled,27,2020-04-14 05:15:26,2610,0,Frenchwhorehouse: bruh blurringthough is gett...,NaN,True,False,False,https://www.reddit.com/r/india/comments/g0lfy8...
1,fx8sbw,Late Night Random Discussion Thread !,"^Beep ^Boop ^Bot, ^I ^am ^a ^bot! ^if ^any ^pr...",False,oxythebot,Scheduled,30,2020-04-09 05:15:11,3362,0,"Called two people today, one was the same pers...",NaN,True,False,False,https://www.reddit.com/r/india/comments/fx8sbw...
2,fzpfrc,"Random Daily Discussion Thread - April 12, 202...","^Beep ^Boop ^Bot, ^I ^am ^a ^bot! ^if ^any ^pr...",False,oxythebot,Scheduled,27,2020-04-12 17:15:29,2073,0,One of my all time favorite movie\n\nhttps://i...,NaN,True,False,False,https://www.reddit.com/r/india/comments/fzpfrc...
3,fzyyju,Late Night Random Discussion Thread !,"^Beep ^Boop ^Bot, ^I ^am ^a ^bot! ^if ^any ^pr...",False,oxythebot,Scheduled,30,2020-04-13 05:15:03,1794,0,Man being alone during this lockdown is soo bo...,NaN,True,False,False,https://www.reddit.com/r/india/comments/fzyyju...
4,fyi8g2,Late Night Random Discussion Thread !,"^Beep ^Boop ^Bot, ^I ^am ^a ^bot! ^if ^any ^pr...",False,oxythebot,Scheduled,29,2020-04-11 05:15:03,2286,0,[deleted],NaN,True,False,False,https://www.reddit.com/r/india/comments/fyi8g2...


Text Preprocessing

In [210]:
rindia.shape

(2878, 16)

In [211]:
rindia.fillna("",inplace = True)
STOPWORDS = set(stopwords.words('english'))

def to_strings(value):
    return str(value)

def preprocess(text):

    text = BeautifulSoup(text, "lxml").text
    text = text.replace("(<br/>)", "")
    text = text.replace('(<a).*(>).*(</a>)', '')
    text = text.replace('(&amp)', '')
    text = text.replace('(&gt)', '')
    text = text.replace('(&lt)', '')
    text = text.replace('(\xa0)', ' ') 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"What's", "what is", text)
    text = re.sub(r"\'s", "", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text=re.sub(r'\b\w{1,2}\b', '', text)
    text=text.lower().replace("indian","india")
    
    return ' '.join(text.split()).replace(r"\s*\([^()]*\)","").strip()

Performing the preprocessing on features like 'title','content','comments' and later generating a synthetic feature by combining all three features + url.

In [212]:
rindia['title'] = rindia['title'].apply(to_strings)
rindia['content'] = rindia['content'].apply(to_strings)
rindia['comments'] = rindia['comments'].apply(to_strings)


rindia['title'] = rindia['title'].apply(preprocess)
rindia['content'] = rindia['content'].apply(preprocess)
rindia['comments'] = rindia['comments'].apply(preprocess)

all_features = rindia["title"] + rindia["comments"] + rindia["content"] + rindia["url"]
rindia['all_features'] = all_features

/home/ashish/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Number of words

In [213]:
rindia['all_features'].apply(lambda x: len(x.split(' '))).sum()

166311

In [214]:
rindia['flair'].unique()

array(['Scheduled', 'AskIndia', '[R]eddiquette', 'Politics',
       'Non-Political', 'Business/Finance', 'Policy/Economy',
       'Science/Technology', 'Sports', 'Food', 'AMA', 'Photography'],
      dtype=object)

Creating a dictionary for the target classes

In [219]:
flair_classes = ['Scheduled', 'AskIndia', '[R]eddiquette', 'Politics',
       'Non-Political', 'Business/Finance', 'Policy/Economy',
       'Science/Technology', 'Sports', 'Food', 'AMA', 'Photography']

Assigning features and target variable

In [220]:
X = rindia.all_features
y = rindia['flair']

Getting the training and testing data

In [221]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

Multinomial NB for Multinomial Models

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
mnb.fit(X_train, y_train)

y_pred = mnb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))  

Logistic Regression

In [140]:
from sklearn.linear_model import LogisticRegression

lr = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                 ])
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flair_classes))

Random Forest Classifier

In [141]:
from sklearn.ensemble import RandomForestClassifier
  
rf = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flair_classes))

Multi-layer perceptron classifier

In [142]:
from sklearn.neural_network import MLPClassifier
  
mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,30,30))),
                 ])
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flair_classes))

In [191]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Conv1D, MaxPooling1D, Flatten
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.models import Sequential

In [171]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

In [154]:
tokenizer = Tokenizer(num_words=None,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ',char_level=False)
tokenizer.fit_on_texts(X_train)

In [155]:
x_train = tokenizer.texts_to_sequences(X_train)

In [156]:
x_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

In [172]:
max_words = 1500
max_len = 200
sequences_matrix = sequence.pad_sequences(x_train,maxlen=max_len)

In [148]:
# f = open('/media/ashish/Seagate Expansion Drive/Crysagi Systems/glove.6B/glove.6B.100d.txt')
# embeddings_index = {}
# for line in f:
#     val = line.split()
#     word = val[0]
#     coff = np.asarray(val[1:],dtype = 'float')
#     embeddings_index[word] = coff

# f.close()
# print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [157]:
# embedding_dim = 100
# k = 0
# embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # Words not found in embedding index will be all-zeros.
#         k += 1
#         embedding_matrix[i] = embedding_vector

In [174]:
embedding_dim = 200

In [223]:
#Sequential LSTM Model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=sequences_matrix.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add((LSTM(200, dropout=0.2, recurrent_dropout=0.2,return_sequences = True))) 
model.add((LSTM(200, dropout=0.2, recurrent_dropout=0.2,return_sequences = True))) 
model.add((LSTM(200, dropout=0.2, recurrent_dropout=0.2))) 
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 200, 200)          300000    
_________________________________________________________________
spatial_dropout1d_12 (Spatia (None, 200, 200)          0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 200, 200)          320800    
_________________________________________________________________
lstm_23 (LSTM)               (None, 200, 200)          320800    
_________________________________________________________________
lstm_24 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dense_41 (Dense)             (None, 256)               51456     
_________________________________________________________________
dense_42 (Dense)             (None, 128)             

In [ ]:
# epochs = 10
# batch_size = 20

# history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

In [224]:
model = Sequential()
model.add(Embedding(max_words, EMBEDDING_DIM, input_length=sequences_matrix.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(nb_filter=16,filter_length=1,border_mode="valid",activation="relu",))
# standard max pooling
model.add(MaxPooling1D(pool_length=2))
model.add(Conv1D(nb_filter=32,filter_length=2,border_mode="valid",activation="relu",))
# standard max pooling
model.add(MaxPooling1D(pool_length=2))  
model.add(Conv1D(nb_filter=64,filter_length=4,border_mode="valid",activation="relu",))
# standard max pooling
model.add(MaxPooling1D(pool_length=2)) 
model.add(Conv1D(nb_filter=128,filter_length=6,border_mode="valid",activation="relu",))
# standard max pooling
model.add(MaxPooling1D(pool_length=2)) 
model.add(Conv1D(nb_filter=256,filter_length=8,border_mode="valid",activation="relu",))
# standard max pooling
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=16, kernel_size=1, padding="valid")`
  after removing the cwd from sys.path.


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 200, 200)          300000    
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 200, 200)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 200, 16)           3216      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 100, 16)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 99, 32)            1056      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 49, 32)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 46, 64)          

/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  
/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=32, kernel_size=2, padding="valid")`
  import sys
/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=4, padding="valid")`
  # Remove the CWD from sys.path while we load stuff.
/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPo

In [ ]:
history = model.fit(X_train, y_train, epochs=15, batch_size=15,validation_split=0.2, verbose=1)

In [ ]:
#Saving the Model
import pickle
filename = 'CNN_model_final.sav'
pickle.dump(model, open(filename, 'wb'))